In [ ]:
# main.py - HOAN CHINH VOI IMPORT
import gradio as gr
import networkx as nx
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import json
import tempfile
import os

# ==================== IMPORT MODULES ====================
from graph_operations import GraphOperations
from utils import draw_graph, adjacency_matrix_to_graph, edge_list_to_graph

# ==================== GLOBAL STATE ====================
current_graph = nx.Graph()
is_directed = False
graph_ops = GraphOperations()  # Instance cua GraphOperations

# ==================== UTILITY FUNCTIONS ====================
def safe_int_convert(val):
    """Chuyen doi an toan sang int"""
    try:
        return int(float(val))
    except:
        return 0

def draw_and_save_graph(G, directed, highlight_path=None, highlight_edges=None, title=""):
    """Vẽ đồ thị và lưu ra file TEMP - FIX PATH TOO LONG"""
    if not G.nodes():
        return None
    
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G, seed=42)
    
    # Cấu hình cơ bản
    node_color = ['lightblue'] * len(G.nodes())
    edge_color = ['gray'] * len(G.edges())
    edge_width = [1] * len(G.edges())
    
    # Highlight PATH (cho Dijkstra, BFS, DFS)
    path_edges = []
    if highlight_path:
        # Tạo danh sách cạnh từ path
        path_edges = [(highlight_path[i], highlight_path[i+1]) 
                     for i in range(len(highlight_path)-1)]
        
        # Highlight nodes trong path
        node_color = ['red' if node in highlight_path else 'lightblue' 
                     for node in G.nodes()]
        
        # Highlight edges trong path
        for i, edge in enumerate(G.edges()):
            if edge in path_edges or (edge[1], edge[0]) in path_edges:
                edge_color[i] = 'red'
                edge_width[i] = 3
    
    # Highlight EDGES cụ thể (cho Prim, Kruskal)
    if highlight_edges:
        for i, edge in enumerate(G.edges()):
            if edge in highlight_edges or (edge[1], edge[0]) in highlight_edges:
                edge_color[i] = 'red'
                edge_width[i] = 3
    
    # Vẽ nodes
    nx.draw_networkx_nodes(G, pos, node_color=node_color, node_size=500)
    nx.draw_networkx_labels(G, pos)
    
    # Vẽ edges với màu sắc và độ dày khác nhau
    if directed:
        nx.draw_networkx_edges(G, pos, edge_color=edge_color, width=edge_width,
                              arrows=True, arrowstyle='-|>', arrowsize=20,
                              connectionstyle='arc3,rad=0.0')
    else:
        # SỬA DÒNG NÀY: BỎ connectionstyle khi không có arrows
        nx.draw_networkx_edges(G, pos, edge_color=edge_color, width=edge_width)
    
    # Thêm trọng số
    edge_labels = nx.get_edge_attributes(G, 'weight')
    if edge_labels:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    
    plt.title(title or f"Đồ thị ({len(G.nodes())} nodes, {len(G.edges())} edges)")
    plt.axis('off')
    
    # Tạo file tạm thời
    temp_dir = tempfile.gettempdir()
    temp_file = os.path.join(temp_dir, "graph_temp.png")
    
    if len(temp_file) > 100:
        temp_file = "C:/temp/graph.png"
    
    plt.savefig(temp_file, bbox_inches='tight', dpi=100)
    plt.close()
    
    return temp_file

# ==================== HANDLERS CO BAN ====================
def create_graph_handler(text, directed):
    """Xu ly tao do thi"""
    global current_graph, is_directed, graph_ops
    
    is_directed = directed
    edges = []
    
    # Xu ly tung dong
    for line in text.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
        
        parts = line.split()
        if len(parts) >= 2:
            try:
                u = safe_int_convert(parts[0])
                v = safe_int_convert(parts[1])
                w = float(parts[2]) if len(parts) > 2 else 1.0
                edges.append((u, v, w))
            except:
                continue
    
    if not edges:
        return "Khong co du lieu hop le", None
    
    # Tao do thi
    current_graph = nx.DiGraph() if directed else nx.Graph()
    for u, v, w in edges:
        current_graph.add_edge(u, v, weight=w)
    
    # Cap nhat graph_ops
    graph_ops.set_graph(current_graph, directed)
    
    img_path = draw_and_save_graph(current_graph, directed, 
                                   title=f"Da tao {len(edges)} canh")
    return f"Tao thanh cong {len(edges)} canh", img_path

def shortest_path_handler(start, end):
    """Tim duong di ngan nhat"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        start = int(start)
        end = int(end)
        
        try:
            path = nx.dijkstra_path(current_graph, start, end)
            length = nx.dijkstra_path_length(current_graph, start, end)
            
            # Tao danh sach canh can highlight
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            
            img_path = draw_and_save_graph(
                current_graph, is_directed, 
                highlight_path=path,
                highlight_edges=path_edges,
                title=f"Duong di ngan nhat: {path} (dai: {length})"
            )
            return f"Duong di: {' -> '.join(map(str, path))}\nDo dai: {length}", img_path
        except nx.NetworkXNoPath:
            img_path = draw_and_save_graph(current_graph, is_directed)
            return "Khong tim thay duong di", img_path
    except:
        img_path = draw_and_save_graph(current_graph, is_directed)
        return "Node khong hop le", img_path

def bfs_handler(start):
    """Xu ly BFS"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        start = int(start)
        # Lay cay BFS
        bfs_tree = nx.bfs_tree(current_graph, start)
        bfs_nodes = list(bfs_tree.nodes())
        
        # Lay cac canh trong cay BFS
        bfs_edges = list(bfs_tree.edges())
        
        img_path = draw_and_save_graph(
            current_graph, is_directed,
            highlight_path=bfs_nodes,
            highlight_edges=bfs_edges,
            title=f"BFS Tree tu node {start}"
        )
        return f"BFS: {bfs_nodes}", img_path
    except:
        img_path = draw_and_save_graph(current_graph, is_directed)
        return "Node khong hop le", img_path

def dfs_handler(start):
    """Xu ly DFS"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        start = int(start)
        # Lay cay DFS
        dfs_tree = nx.dfs_tree(current_graph, start)
        dfs_nodes = list(dfs_tree.nodes())
        
        # Lay cac canh trong cay DFS
        dfs_edges = list(dfs_tree.edges())
        
        img_path = draw_and_save_graph(
            current_graph, is_directed,
            highlight_path=dfs_nodes,
            highlight_edges=dfs_edges,
            title=f"DFS Tree tu node {start}"
        )
        return f"DFS: {dfs_nodes}", img_path
    except:
        img_path = draw_and_save_graph(current_graph, is_directed)
        return "Node khong hop le", img_path

def bipartite_handler():
    """Kiem tra do thi 2 phia - Dung GraphOperations"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        # Dung graph_ops
        is_bip = graph_ops.is_bipartite()
        result = "La do thi 2 phia" if is_bip else "Khong phai do thi 2 phia"
        img_path = draw_and_save_graph(current_graph, is_directed, title=result)
        return result, img_path
    except:
        img_path = draw_and_save_graph(current_graph, is_directed)
        return "Khong the kiem tra", img_path

# ==================== HANDLERS NANG CAO ====================
def prim_handler():
    """Thuat toan Prim"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        mst_edges = graph_ops.prim_mst()
        if mst_edges:
            total_weight = sum(weight for _, _, weight in mst_edges)
            
            # Lay danh sach canh MST de highlight
            mst_edge_list = [(u, v) for u, v, _ in mst_edges]
            
            img_path = draw_and_save_graph(
                current_graph, is_directed,
                highlight_edges=mst_edge_list,
                title=f"Prim MST - Tong trong so: {total_weight}"
            )
            result = f"Cay khung nho nhat (Prim):\n"
            for u, v, w in mst_edges:
                result += f"  {u} -> {v} (w={w})\n"
            result += f"Tong trong so: {total_weight}"
            return result, img_path
        else:
            return "Do thi khong lien thong", None
    except Exception as e:
        return f"Loi: {str(e)}", None

def kruskal_handler():
    """Thuat toan Kruskal"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        mst_edges = graph_ops.kruskal_mst()
        if mst_edges:
            total_weight = sum(weight for _, _, weight in mst_edges)
            
            # Lay danh sach canh MST de highlight
            mst_edge_list = [(u, v) for u, v, _ in mst_edges]
            
            img_path = draw_and_save_graph(
                current_graph, is_directed,
                highlight_edges=mst_edge_list,
                title=f"Kruskal MST - Tong trong so: {total_weight}"
            )
            result = f"Cay khung nho nhat (Kruskal):\n"
            for u, v, w in mst_edges:
                result += f"  ({u}, {v}) - {w}\n"
            result += f"Tong trong so: {total_weight}"
            return result, img_path
        else:
            return "Do thi khong lien thong", None
    except Exception as e:
        return f"Loi: {str(e)}", None

def ford_fulkerson_handler(source, sink):
    """Thuat toan Ford-Fulkerson"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    try:
        source = int(source)
        sink = int(sink)
        
        max_flow = graph_ops.ford_fulkerson(source, sink)
        
        img_path = draw_and_save_graph(
            current_graph, is_directed,
            title=f"Ford-Fulkerson - Luong cuc dai: {max_flow}"
        )
        
        return f"Luong cuc dai tu {source} -> {sink}: {max_flow}", img_path
    except Exception as e:
        return f"Loi: {str(e)}", None

def fleury_handler(start_node):
    """Thuật toán Fleury (tìm chu trình Euler) - Dùng GraphOperations"""
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    
    try:
        start = int(start_node)
        # Dùng graph_ops thay vì NetworkX trực tiếp
        circuit = graph_ops.fleury_eulerian_path(start)
        
        if circuit:
            # Chuyển circuit thành các cạnh để highlight
            circuit_edges = list(circuit)
            
            # Tạo path từ circuit để highlight nodes
            path = [start]
            for u, v in circuit:
                if v not in path:
                    path.append(v)
            
            img_path = draw_and_save_graph(
                current_graph, is_directed,
                highlight_path=path,
                highlight_edges=circuit_edges,
                title=f"Fleury - Chu trình Euler"
            )
            
            result = f"✅ Chu trình Euler (Fleury):\n"
            for u, v in circuit:
                result += f"  {u} → {v}\n"
            return result, img_path
        
        return "⚠ Đồ thị không có chu trình Euler", None
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def hierholzer_handler(start_node):
    """Thuật toán Hierholzer (tìm chu trình Euler) - Dùng GraphOperations"""
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    
    try:
        start = int(start_node)
        # Dùng graph_ops thay vì NetworkX trực tiếp
        circuit = graph_ops.hierholzer_eulerian_circuit(start)
        
        if circuit:
            # Chuyển circuit thành các cạnh để highlight
            circuit_edges = list(circuit)
            
            img_path = draw_and_save_graph(
                current_graph, is_directed,
                highlight_edges=circuit_edges,
                title=f"Hierholzer - Chu trình Euler"
            )
            
            result = f"✅ Chu trình Euler (Hierholzer):\n"
            for u, v in circuit:
                result += f"  {u} → {v}\n"
            return result, img_path
        
        return "⚠ Đồ thị không có chu trình Euler", None
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def advanced_algo_handler(algo_choice, param1=None, param2=None):
    """Xu ly thuat toan nang cao"""
    if not current_graph.nodes():
        return "Chua co do thi", None
    
    if algo_choice == "Prim":
        return prim_handler()
    elif algo_choice == "Kruskal":
        return kruskal_handler()
    elif algo_choice == "Ford-Fulkerson":
        if param1 is None or param2 is None:
            return "Vui long nhap source va sink", None
        return ford_fulkerson_handler(param1, param2)
    elif algo_choice == "Fleury":
        if param1 is None:
            return "Vui long nhap node bat dau", None
        return fleury_handler(param1)
    elif algo_choice == "Hierholzer":
        if param1 is None:
            return "Vui long nhap node bat dau", None
        return hierholzer_handler(param1)
    
    return "Vui long chon thuat toan", None

# ==================== GRADIO UI ====================
with gr.Blocks(title="Graph Visualizer", theme=gr.themes.Soft()) as demo:
    
    # Header
    gr.Markdown("# TRINH XU LY DO THI")
    gr.Markdown("Nhap do thi va thuc hien cac thuat toan co ban & nang cao")
    
    with gr.Tabs():
        # TAB 1: NHAP DO THI
        with gr.Tab("Nhap do thi"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Nhap danh sach canh")
                    input_text = gr.Textbox(
                        label="Moi dong: u v [weight]",
                        placeholder="Vi du:\n0 1 5\n0 2 3\n1 2 2",
                        lines=10,
                        value="0 1 5\n0 2 3\n1 2 2"
                    )
                    
                    with gr.Row():
                        directed_cb = gr.Checkbox(label="Do thi co huong", value=False)
                        create_btn = gr.Button("Tao do thi", variant="primary", size="lg")
                    
                    status = gr.Textbox(label="Trang thai", interactive=False)
                
                with gr.Column(scale=1):
                    gr.Markdown("### Hien thi")
                    output_img = gr.Image(label="Do thi")
            
            create_btn.click(
                fn=create_graph_handler,
                inputs=[input_text, directed_cb],
                outputs=[status, output_img]
            )
        
        # TAB 2: THUAT TOAN CO BAN
        with gr.Tab("Thuat toan co ban"):
            with gr.Row():
                with gr.Column():
                    # Dijkstra
                    gr.Markdown("### Duong di ngan nhat")
                    with gr.Row():
                        start_node = gr.Number(label="Node bat dau", value=0, precision=0)
                        end_node = gr.Number(label="Node ket thuc", value=1, precision=0)
                    
                    dijkstra_btn = gr.Button("Tim duong di", variant="primary")
                    dijkstra_result = gr.Textbox(label="Ket qua")
                    
                    # BFS/DFS
                    gr.Markdown("### Duyet do thi")
                    traversal_start = gr.Number(label="Node bat dau", value=0, precision=0)
                    
                    with gr.Row():
                        bfs_btn = gr.Button("BFS")
                        dfs_btn = gr.Button("DFS")
                    
                    traversal_result = gr.Textbox(label="Ket qua duyet")
                    
                    # Bipartite
                    gr.Markdown("### Kiem tra tinh chat")
                    bipartite_btn = gr.Button("Kiem tra 2 phia")
                    bipartite_result = gr.Textbox(label="Ket qua")
                
                with gr.Column():
                    algo_img = gr.Image(label="Ket qua truc quan")
            
            # Ket noi su kien
            dijkstra_btn.click(
                fn=shortest_path_handler,
                inputs=[start_node, end_node],
                outputs=[dijkstra_result, algo_img]
            )
            
            bfs_btn.click(
                fn=bfs_handler,
                inputs=[traversal_start],
                outputs=[traversal_result, algo_img]
            )
            
            dfs_btn.click(
                fn=dfs_handler,
                inputs=[traversal_start],
                outputs=[traversal_result, algo_img]
            )
            
            bipartite_btn.click(
                fn=bipartite_handler,
                outputs=[bipartite_result, algo_img]
            )
        
        # TAB 3: THUAT TOAN NANG CAO
        with gr.Tab("Thuat toan nang cao"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Lua chon thuat toan")
                    
                    algo_choice = gr.Radio(
                        choices=["Prim", "Kruskal", "Ford-Fulkerson", "Fleury", "Hierholzer"],
                        label="Chon thuat toan",
                        value="Prim"
                    )
                    
                    # Dynamic inputs based on algorithm
                    with gr.Group() as param_group:
                        source_input = gr.Number(
                            label="Source node (cho Ford-Fulkerson)",
                            value=0,
                            precision=0,
                            visible=False
                        )
                        sink_input = gr.Number(
                            label="Sink node (cho Ford-Fulkerson)",
                            value=1,
                            precision=0,
                            visible=False
                        )
                        start_input = gr.Number(
                            label="Node bat dau (cho Fleury/Hierholzer)",
                            value=0,
                            precision=0,
                            visible=False
                        )
                    
                    # Update input visibility based on algorithm choice
                    def update_inputs(algo):
                        vis_source = (algo == "Ford-Fulkerson")
                        vis_sink = (algo == "Ford-Fulkerson")
                        vis_start = (algo in ["Fleury", "Hierholzer"])
                        
                        return [
                            gr.update(visible=vis_source),
                            gr.update(visible=vis_sink),
                            gr.update(visible=vis_start)
                        ]
                    
                    algo_choice.change(
                        fn=update_inputs,
                        inputs=[algo_choice],
                        outputs=[source_input, sink_input, start_input]
                    )
                    
                    run_algo_btn = gr.Button("Chay thuat toan", variant="primary", size="lg")
                
                with gr.Column(scale=1):
                    gr.Markdown("### Ket qua")
                    advanced_result = gr.Textbox(label="Ket qua thuat toan", lines=6)
                    advanced_img = gr.Image(label="Truc quan hoa")
            
            # Handler cho nut chay thuat toan
            def run_advanced_algo(algo, source, sink, start):
                if algo == "Ford-Fulkerson":
                    return advanced_algo_handler(algo, source, sink)
                elif algo in ["Fleury", "Hierholzer"]:
                    return advanced_algo_handler(algo, start, None)
                else:
                    return advanced_algo_handler(algo, None, None)
            
            run_algo_btn.click(
                fn=run_advanced_algo,
                inputs=[algo_choice, source_input, sink_input, start_input],
                outputs=[advanced_result, advanced_img]
            )
        
        # TAB 4: CHUYEN DOI
        with gr.Tab("Chuyen doi"):
            gr.Markdown("### Chuyen doi bieu dien")
            
            format_type = gr.Radio(
                choices=["Ma tran ke", "Danh sach ke", "Danh sach canh"],
                label="Chon dinh dang",
                value="Danh sach canh"
            )
            
            convert_btn = gr.Button("Chuyen doi", variant="primary")
            conversion_output = gr.Textbox(label="Ket qua", lines=10)
            
            def convert_handler(format_type):
                if not current_graph.nodes():
                    return "Chua co do thi"
                
                try:
                    if format_type == "Ma tran ke":
                        # Dung graph_ops
                        matrix, nodes = graph_ops.to_adjacency_matrix()
                        result = "Ma tran ke:\n"
                        result += "   " + " ".join(str(n) for n in nodes) + "\n"
                        for i, row in enumerate(matrix):
                            result += f"{nodes[i]}: " + " ".join(str(x) for x in row) + "\n"
                        
                    elif format_type == "Danh sach ke":
                        # Dung graph_ops
                        adj_list = graph_ops.to_adjacency_list()
                        result = "Danh sach ke:\n"
                        for node in sorted(adj_list.keys()):
                            neighbors = adj_list[node]
                            neighbor_str = ", ".join(f"{n}({w})" for n, w in neighbors)
                            result += f"{node}: {neighbor_str}\n"
                    
                    else:  # Danh sach canh
                        # Dung graph_ops
                        edges = graph_ops.to_edge_list()
                        result = "Danh sach canh:\n"
                        for u, v, w in edges:
                            result += f"({u}, {v}, {w})\n"
                    
                    return result
                except Exception as e:
                    return f"Loi: {str(e)}"
            
            convert_btn.click(
                fn=convert_handler,
                inputs=[format_type],
                outputs=[conversion_output]
            )
        
        # TAB 5: LUU/TAI
        with gr.Tab("Luu/Tai"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Luu do thi")
                    save_btn = gr.Button("Xuat JSON", variant="primary")
                    json_output = gr.Textbox(label="Du lieu JSON", lines=8)
                    
                    def save_handler():
                        if not current_graph.nodes():
                            return "Chua co do thi"
                        
                        edges = [(u, v, current_graph[u][v].get('weight', 1)) 
                                for u, v in current_graph.edges()]
                        
                        data = {
                            "directed": is_directed,
                            "nodes": list(current_graph.nodes()),
                            "edges": edges
                        }
                        return json.dumps(data, indent=2)
                    
                    save_btn.click(fn=save_handler, outputs=[json_output])
                
                with gr.Column():
                    gr.Markdown("### Tai do thi")
                    json_input = gr.Textbox(
                        label="Dan JSON o day",
                        placeholder='{"directed": false, "edges": [[0,1,5], [0,2,3]]}',
                        lines=8
                    )
                    
                    load_btn = gr.Button("Tai tu JSON")
                    load_status = gr.Textbox(label="Trang thai")
                    
                    def load_handler(json_str):
                        try:
                            data = json.loads(json_str)
                            global current_graph, is_directed, graph_ops
                            
                            is_directed = data.get("directed", False)
                            current_graph = nx.DiGraph() if is_directed else nx.Graph()
                            
                            for u, v, w in data.get("edges", []):
                                current_graph.add_edge(u, v, weight=w)
                            
                            # Cap nhat graph_ops
                            graph_ops.set_graph(current_graph, is_directed)
                            
                            img_path = draw_and_save_graph(current_graph, is_directed,
                                                         title="Do thi da tai")
                            return "Da tai thanh cong", img_path
                        except:
                            return "JSON khong hop le", None
                    
                    load_btn.click(
                        fn=load_handler,
                        inputs=[json_input],
                        outputs=[load_status, output_img]
                    )
    
    # Footer
    gr.Markdown("---")
    gr.Markdown("""
    ### HUONG DAN NHANH:
    1. **Tab 1**: Nhap do thi (moi dong: `u v weight`)
    2. **Tab 2**: Thuat toan co ban (Dijkstra, BFS, DFS, 2 phia)
    3. **Tab 3**: Thuat toan nang cao (Prim, Kruskal, Ford-Fulkerson, Fleury, Hierholzer)
    4. **Tab 4**: Chuyen doi dinh dang
    5. **Tab 5**: Luu/tai do thi
    """)

# ==================== CHAY UNG DUNG ====================
if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",
        server_port=7875,
        share=False,
        show_error=True
    )

* Running on local URL:  http://0.0.0.0:7875
* To create a public link, set `share=True` in `launch()`.


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13180\735056108.py:75: UserWarning: 

The connectionstyle keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  nx.draw_networkx_edges(G, pos, edge_color=edge_color, width=edge_width, connectionstyle='arc3,rad=0.0')
